<a href="https://colab.research.google.com/github/marcelo-find-ai/logos/blob/main/Salvar_dados_delta_bronze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
(base) PS C:\Users\mt> aws s3 ls s3://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/sinistro/
File association not found for extension .py
2024-11-21 15:39:13  459169597 ID50056_201801 Sinistro.csv
2024-11-21 15:41:15  604538624 ID50056_201802 Sinistro.csv
2024-11-21 15:41:33  591872417 ID50056_201803 Sinistro.csv
2024-11-21 15:40:57  584321565 ID50056_201804 Sinistro.csv
2024-11-21 15:41:48  628012956 ID50056_201805 Sinistro.csv
2024-11-21 15:49:57  617011374 ID50056_201806 Sinistro.csv
2024-11-21 15:50:34  554080371 ID50056_201807 Sinistro.csv
2024-11-21 15:52:40  645584312 ID50056_201808 Sinistro.csv
2024-11-21 15:52:03  591070497 ID50056_201809 Sinistro.csv
2024-11-21 15:52:19  603318683 ID50056_201810 Sinistro.csv
2024-11-21 17:23:32  610442913 ID50056_201811 Sinistro.csv
2024-11-21 17:24:18  586624830 ID50056_201812 Sinistro.csv
2024-11-21 17:26:03  464721446 ID50056_201901 Sinistro.csv
2024-11-21 17:26:13  575585600 ID50056_201902 Sinistro.csv
2024-11-21 17:25:32  568262354 ID50056_201903 Sinistro.csv
2024-11-21 17:34:35  628106245 ID50056_201904 Sinistro.csv
2024-11-21 17:35:11  630511107 ID50056_201905 Sinistro.csv
2024-11-21 17:36:56  663403654 ID50056_201906 Sinistro.csv
2024-11-21 17:37:45  670524627 ID50056_201907 Sinistro.csv
2024-11-21 17:38:53  728752508 ID50056_201908 Sinistro.csv
2024-11-21 17:45:37  645090420 ID50056_201909 Sinistro.csv
2024-11-21 17:47:37  723382446 ID50056_201910 Sinistro.csv
2024-11-21 17:48:33  662425771 ID50056_201911 Sinistro.csv
2024-11-21 17:48:48  636197938 ID50056_201912 Sinistro.csv
2024-11-21 17:45:50  392148481 ID55228_Parte_III_201801.csv
2024-11-21 17:54:47  529513688 ID55228_Parte_III_201802.csv
2024-11-21 17:54:41  503006597 ID55228_Parte_III_201803.csv
2024-11-21 17:56:31  502362552 ID55228_Parte_III_201804.csv
2024-11-21 17:58:08  549809734 ID55228_Parte_III_201805.csv
2024-11-21 17:58:21  542951370 ID55228_Parte_III_201806.csv
2024-11-21 18:03:04  478883233 ID55228_Parte_III_201807.csv
2024-11-21 18:04:43  567380396 ID55228_Parte_III_201808.csv
2024-11-21 18:05:30  517525006 ID55228_Parte_III_201809.csv
2024-11-21 18:07:15  522505670 ID55228_Parte_III_201810.csv
2024-11-21 18:07:33  530312337 ID55228_Parte_III_201811.csv
2024-11-21 18:12:20  523018266 ID55228_Parte_III_201812.csv
2024-11-21 18:11:53  396955670 ID55228_Parte_III_201901.csv
2024-11-21 18:14:38  507204014 ID55228_Parte_III_201902.csv
2024-11-21 18:16:22  494823665 ID55228_Parte_III_201903.csv
2024-11-21 18:36:35  556644818 ID55228_Parte_III_201904.csv
2024-11-21 18:40:49  556886319 ID55228_Parte_III_201905.csv
2024-11-21 18:41:54  592714854 ID55228_Parte_III_201906.csv
2024-11-21 18:44:01  590406213 ID55228_Parte_III_201907.csv
2024-11-21 18:46:38  643607032 ID55228_Parte_III_201908.csv
2024-11-21 18:46:38  564517872 ID55228_Parte_III_201909.csv
2024-11-21 18:51:52  638280426 ID55228_Parte_III_201910.csv
2024-11-21 18:52:05  581697401 ID55228_Parte_III_201911.csv
2024-11-21 18:53:37  568161588 ID55228_Parte_III_201912.csv
2024-11-21 18:55:03  546688731 T60866_202201 Sinistro.csv
2024-11-21 18:56:01  680288679 T60866_202202 Sinistro.csv
2024-11-21 18:57:11  713244036 T60866_202203 Sinistro.csv
```

In [1]:
# Instalar a lib
!pip install pyspark==3.1.2 --quiet

In [2]:
# IMPORTANTE:
# Configurar aqui as variáveis de ambiente para acesso ao datalake Find AI
import os
from google.colab import userdata


os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('aws_access_key_id')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('aws_secret_access_key')
os.environ['AWS_DEFAULT_REGION'] = userdata.get('region_name')
os.environ['AWS_ENDPOINT_URL'] = userdata.get('endpoint_url')

In [3]:
import pyspark

In [4]:
spark = (pyspark.sql.SparkSession
    .builder
    .appName('datasus')
    .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk-bundle:1.11.819,org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:1.0.1')
    .config("spark.driver.memory", "8G")
    .config("spark.hadoop.fs.s3a.endpoint", os.environ['AWS_ENDPOINT_URL'])
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.committer.name", "directory")
    .config("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")
    .config("spark.hadoop.mapreduce.reduce.speculative","false")
    .config("spark.hadoop.fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID'])
    .config("spark.hadoop.fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY'] )
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
    .getOrCreate())

In [ ]:
# Lendo dados do datalake
dados =        "s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/sinistro/ID50056_*.csv"
delta_output = "s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/sinistro/dataset_id=ID50056"

In [ ]:
df = (spark.read
      .option("header", "true")
      .option("sep", "|^")
      .option("inferSchema", "false")
      .option("encoding", "ISO-8859-1")
      .csv(dados)
      #.cache()
      )

In [ ]:
df.show()

+-----------+-------------+--------------------+-----------------+--------------+--------------------+-----------------+--------------------+---------+----------------+----------------+---------+---------------+-------------------+--------------+---------------+-------------+-------+--------------+--------------------+------------+----------------+--------------------+---------------------+----------------------+-------------------------+---------------------------+------------------+---------------+--------------+--------------+--------------------+--------------------+-----------------+-------------+-----------------+----+---------------+-----------+--------------------+-----------------+----------------+-----------+--------------------+--------------------+------------+-------------+-------------+--------------+-------------+--------+---------+--------------+---------------------+-------------+
|competencia|nr_guia_princ|             nr_guia|senha_solicitacao|cod_prest_capa|     nom

In [ ]:
delta_output

's3a://find-ai-org-data/delta/bronze/unimed_florianopolis/sinistro/dataset_id=ID50056'

In [ ]:
# Write df in s3a as delta
df\
    .write\
    .format("delta")\
    .mode('overwrite')\
    .option("overwriteSchema", "true")\
    .save(delta_output)

In [ ]:
# Test
data = spark.read.format("delta").load(delta_output)

In [ ]:
data.show()

In [ ]:
data.count()

20336482

In [7]:
### Dados de usuarios:

In [13]:
!aws s3 ls s3://find-ai-org-data/delta/bronze/unimed_florianopolis/sinistro/

                           PRE dataset_id=ID50056/
                           PRE dataset_id=ID55228/
                           PRE dataset_id=T60866/


In [5]:
dados = [
    {
        "input":  "s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/beneficiarios/ID50052 - Dados Projeto de Dimensionamento da Rede Prestadora - Relatório 3 - parte II_ATIVO_envio.csv",
        "output": "s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/beneficiarios/dataset_id=ID50052/tabela=parte II_ATIVO_envio",
        "sep": "|^",
    },
    {
        "input":  "s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/beneficiarios/ID50052 - Dados Projeto de Dimensionamento da Rede Prestadora - Relatório 3 - parte II_DIFERENTE_DE_ATIVO_envio.csv",
        "output": "s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/beneficiarios/dataset_id=ID50052/tabela=parte II_DIFERENTE_DE_ATIVO_envio",
        "sep": "|^",
    },
    {
        "input":  "s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/beneficiarios/T60862 - Dados Projeto de Dimensionamento da Rede Prestadora - Relatório 3 - Beneficiários_ATIVO_ANONIMIZADO.csv",
        "output": "s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/beneficiarios/dataset_id=T60862/tabela=Beneficiários_ATIVO_ANONIMIZADO",
        "sep": ";",
    },
    {
        "input":  "s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/beneficiarios/T60862 - Dados Projeto de Dimensionamento da Rede Prestadora - Relatório 3 - Beneficiários_DIFERENTE DE ATIVO_ANONIMIZADO.csv",
        "output": "s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/beneficiarios/dataset_id=T60862/tabela=Beneficiários_DIFERENTE DE ATIVO_ANONIMIZADO",
        "sep": ";",
    },
]

In [65]:
# Usar isso para testar se os outputs estão ok

df = spark.read\
      .option("header", "true")\
      .option("sep", dados[0]["sep"])\
      .option("inferSchema", "false")\
      .option("encoding", "ISO-8859-1")\
      .csv(dados[0]["input"])\
      .toPandas()

In [66]:
df.columns

Index([' NOVO_COD_BENEFICIARIO', 'DATA_NASCIMENTO', 'GENERO', 'DEPENDENCIA',
       'DATA_INCLUSAO', 'DATA_EXCLUSAO', 'SITUACAO', 'NOVO_COD_TITUTLAR',
       'TIPO_ENDERECO', 'FINALIDADE', 'LOGRADOURO', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP', 'TIPO_PRECO', 'IND_FUNDACAO',
       'TIPO_CONTRATACAO', 'NUM_EMPRESA', 'NOME_EMPRESA', 'NUM_CONTRATO',
       'PLANO_ABRANGENCIA', 'PLANO_ACOMODACAO '],
      dtype='object')

In [71]:
spark.createDataFrame(df.to_dict("records")).show()

+----------------------+--------------------+--------+--------------------+--------------------+-------------+-------------+---------------+-------------+------+---------------+------+------------+--------------------+--------------------+-----------------+------+------------+-----------+-------------------+-----------------+--------+--------------------+-------------------+--------------------+
| NOVO_COD_BENEFICIARIO|              BAIRRO|     CEP|              CIDADE|         COMPLEMENTO|DATA_EXCLUSAO|DATA_INCLUSAO|DATA_NASCIMENTO|  DEPENDENCIA|ESTADO|     FINALIDADE|GENERO|IND_FUNDACAO|          LOGRADOURO|        NOME_EMPRESA|NOVO_COD_TITUTLAR|NUMERO|NUM_CONTRATO|NUM_EMPRESA|  PLANO_ABRANGENCIA|PLANO_ACOMODACAO |SITUACAO|    TIPO_CONTRATACAO|      TIPO_ENDERECO|          TIPO_PRECO|
+----------------------+--------------------+--------+--------------------+--------------------+-------------+-------------+---------------+-------------+------+---------------+------+------------+-----

In [6]:
# Rodar ETL em todos os beneficiarios:

# Marcador de conserto do dado:
consertar = False

for paths in dados:
    # ler o dataset e dinamicamente salvar em delta

    print(f"processando input:\n{paths['input']}")

    d = spark.read\
      .option("header", "true")\
      .option("sep", paths["sep"])\
      .option("inferSchema", "false")\
      .option("encoding", "ISO-8859-1")\
      .csv(paths["input"])

    # Corrigir colunas que não são compatíveis com Delta:
    if (' COD_BENEFICIARIO' in d.columns) | ('PLANO_ACOMODACAO ' in d.columns):
        print("dados com defeito - consertando colunas ruins...")
        d = d.toPandas()
        consertar = True

    if consertar:
        if ' NOVO_COD_BENEFICIARIO' in d.columns:
            d[' NOVO_COD_BENEFICIARIO'] = d[' NOVO_COD_BENEFICIARIO'].str.strip()

        if 'PLANO_ACOMODACAO ' in d.columns:
            d['PLANO_ACOMODACAO '] = d['PLANO_ACOMODACAO '].str.strip()

        d.columns = [col.strip() for col in d.columns]

        spark.createDataFrame(d.to_dict("records")).write\
            .format("delta")\
            .mode('overwrite')\
            .option("overwriteSchema", "true")\
            .save(paths["output"])
    else:
        d.write\
            .format("delta")\
            .mode('overwrite')\
            .option("overwriteSchema", "true")\
            .save(paths["output"])

    print(f"dados salvos em:\n{paths['output']}")

    # setar "consertar" para False de novo
    consertar = False

processando input:
s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/beneficiarios/ID50052 - Dados Projeto de Dimensionamento da Rede Prestadora - Relatório 3 - parte II_ATIVO_envio.csv
dados com defeito - consertando colunas ruins...
dados salvos em:
s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/beneficiarios/dataset_id=ID50052/tabela=parte II_ATIVO_envio
processando input:
s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/beneficiarios/ID50052 - Dados Projeto de Dimensionamento da Rede Prestadora - Relatório 3 - parte II_DIFERENTE_DE_ATIVO_envio.csv
dados com defeito - consertando colunas ruins...
dados salvos em:
s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/beneficiarios/dataset_id=ID50052/tabela=parte II_DIFERENTE_DE_ATIVO_envio
processando input:
s3a://find-ai-org-data/datasets/dados_teste/bronze/unimed_florianopolis/beneficiarios/T60862 - Dados Projeto de Dimensionamento da Rede Prestadora - Relatório 3 - Benefici

In [9]:
# testando output:
spark.read.format("delta")\
.load("s3a://find-ai-org-data/delta/bronze/unimed_florianopolis/beneficiarios/dataset_id=T60862/tabela=Beneficiários_DIFERENTE DE ATIVO_ANONIMIZADO")\
.show()

+---------------------+---------------+------+-------------+-------------+-------------+--------+-----------------+-------------+----------+----------+------+-----------+------+------+------+----+--------------------+------------+--------------------+-----------+--------------------+------------+-------------------+----------------+
|NOVO_COD_BENEFICIARIO|DATA_NASCIMENTO|GENERO|  DEPENDENCIA|DATA_INCLUSAO|DATA_EXCLUSAO|SITUACAO|NOVO_COD_TITUTLAR|TIPO_ENDERECO|FINALIDADE|LOGRADOURO|NUMERO|COMPLEMENTO|BAIRRO|CIDADE|ESTADO| CEP|          TIPO_PRECO|IND_FUNDACAO|    TIPO_CONTRATACAO|NUM_EMPRESA|        NOME_EMPRESA|NUM_CONTRATO|  PLANO_ABRANGENCIA|PLANO_ACOMODACAO|
+---------------------+---------------+------+-------------+-------------+-------------+--------+-----------------+-------------+----------+----------+------+-----------+------+------+------+----+--------------------+------------+--------------------+-----------+--------------------+------------+-------------------+-------------